In [ ]:
! mkdir card

In [ ]:
!pip install unrar
!unrar x "/content/drive/MyDrive/Copy of cardamom.rar" "/content/card"


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/drive/MyDrive/Copy of cardamom.rar

Creating    /content/card/cardamom                                    OK
Creating    /content/card/cardamom/healthy                            OK
Extracting  /content/card/cardamom/healthy/HEALTHY0.jpg                    0%  OK 
Extracting  /content/card/cardamom/healthy/HEALTHY1.jpg                    0%  OK 
Extracting  /content/card/cardamom/healthy/HEALTHY10.jpg                   0%  OK 
Extracting  /content/card/cardamom/healthy/HEALTHY100.jpg                  0%  OK 
Extracting  /content/card/cardamom/healthy/HEALTHY101.jpg                  0%  OK 
Extracting  /content/card/cardamom/healthy/HEALTHY102.jpg                  0%  OK 
Extracting  /content/card/cardamom/healthy/HEALTHY103.jpg                  0%  OK 
Extracting  /content/card/cardamom/healthy/HEALTHY104.jpg                  0

In [ ]:
!pip install split-folders
import splitfolders

In [ ]:
! mkdir dataset

In [ ]:
splitfolders.ratio("/content/card/cardamom", output="/content/dataset", seed=1337, ratio=(.8, .1, .1), group_prefix=None) # default values

Copying files: 900 files [00:00, 4428.73 files/s]


In [ ]:
!nvidia-smi

Tue Feb  9 15:27:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install tensorflow-gpu

     |████████████████████████████████| 394.3MB 40kB/s 


In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [ ]:

# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/dataset/train'
valid_path = '/content/dataset/val'

In [ ]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:

# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [ ]:
# useful for getting number of output classes
folders = glob('/content/dataset/train/*')

In [ ]:
folders

['/content/dataset/train/nimiton',
 '/content/dataset/train/healthy',
 '/content/dataset/train/mossic']

In [ ]:
# our layers - you can add more if you want
x = Flatten()(inception.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 111, 111, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 111, 111, 32) 0           batch_normalization_94[0][0]     
____________________________________________________________________________________________

In [ ]:

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

#AUGMENTATION 1


In [ ]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

#AUGMENTATION 2


In [ ]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                  rotation_range=20,
		                              zoom_range=0.15,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.15,
                                  horizontal_flip=True,
                                  fill_mode="nearest")
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/dataset/train/',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 720 images belonging to 3 classes.


In [ ]:
labels = (training_set.class_indices)
print(labels)

{'healthy': 0, 'mossic': 1, 'nimiton': 2}


In [ ]:
test_set = test_datagen.flow_from_directory('/content/dataset/val',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 90 images belonging to 3 classes.


In [ ]:
labels = (test_set.class_indices)
print(labels)

{'healthy': 0, 'mossic': 1, 'nimiton': 2}


In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=4,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/4
23/23 [==============================] - 17s 528ms/step - loss: 6.6765 - accuracy: 0.4457 - val_loss: 0.6440 - val_accuracy: 0.8333
Epoch 2/4
23/23 [==============================] - 11s 454ms/step - loss: 0.9471 - accuracy: 0.8477 - val_loss: 0.2717 - val_accuracy: 0.9556
Epoch 3/4
23/23 [==============================] - 11s 456ms/step - loss: 0.2060 - accuracy: 0.9457 - val_loss: 0.1013 - val_accuracy: 0.9667
Epoch 4/4
23/23 [==============================] - 11s 457ms/step - loss: 0.1373 - accuracy: 0.9608 - val_loss: 0.0534 - val_accuracy: 0.9778


In [ ]:
from keras.models import load_model

model.save('/content/drive/MyDrive/Final models/cardamom/Cardamom_INCEPTION.h5')  

In [ ]:
from keras.models import load_model

model=load_model('/content/drive/MyDrive/Final models/cardamom/Cardamom_INCEPTION.h5')  

In [ ]:
import os
from google.colab.patches import cv2_imshow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
disease= ['healthy', 'mossic', 'nimiton']
org_disease= ['healthy', 'mosaic', 'nimiton']
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.image as mpimg
#img = mpimg.imread('your_image.png')

dir_path="/content/dataset/val/"
k=0
count=0
for y in range(0, 3):
  print(org_disease[y])
  for i in os.listdir(dir_path+disease[y]):
    k=k+1
    img = image.load_img(dir_path+disease[y]+"/"+i, target_size=(224, 224))
    #imgplot = plt.imshow(img)
    #plt.show()
    # Preprocessing the image
    x = image.img_to_array(img)
    # x = np.true_divide(x, 255)
    ## Scaling
    x=x/255
    x = np.expand_dims(x, axis=0)
    preds = model.predict(x)
    if org_disease[np.argmax(preds)] ==str(org_disease[y]):
      count=count+1
    #print(org_disease[np.argmax(preds)])
  print("total images :"+str(k))
  print("Correctly predicted :"+str(count))
  print(str(float(count/k)*float(100))+ "%") 
  print("")

healthy
total images :30
Correctly predicted :30
100.0%

mosaic
total images :60
Correctly predicted :58
96.66666666666667%

nimiton
total images :90
Correctly predicted :88
97.77777777777777%



In [ ]:
import numpy as np
import os
from google.colab.patches import cv2_imshow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
disease= ['healthy', 'mosaic', 'nimiton']
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
img = image.load_img("/content/bdt.png", target_size=(224, 224))
#imgplot = plt.imshow(img)
#plt.show()content/nnn.png",
# Preprocessing the image
x = image.img_to_array(img)
# x = np.true_divide(x, 255)
## Scaling
x=x/255
x = np.expand_dims(x, axis=0)
preds = model.predict(x)
print(disease[np.argmax(preds)])


FileNotFoundError: ignored

In [ ]:
sum = np.squeeze(model.predict(x))
val = 0
for i in range(0, len(sum)):
  val = val + sum[i]
print(val)
print(sum[1] * 100)

1.0000000007398204
1.3370195373238437e-14


In [ ]:
np.squeeze(model.predict(x))

array([7.2988254e-10, 1.3370195e-16, 1.1978398e-21, 1.0000000e+00,
       3.0726707e-15, 9.9287687e-12, 5.8922602e-15], dtype=float32)